In [1]:
import SimpleITK as sitk
import pandas as pd
import os
import numpy as np

In [2]:
#
# open structure properties file
#
csv_file = 'structure_properties.csv'
df = pd.read_csv(csv_file)
df.set_index('acronym',inplace=True)

In [3]:
#
# open annotation file
#
model_directory = "C:/Users/lydia/OneDrive/Work/DigitalAssets/_UPENN_fMOST/"
annotation_file = os.path.join( model_directory,  "25um_V3", "CCFTemplate25umLabels.nii.gz")

annotation = sitk.ReadImage( annotation_file )

In [4]:
def create_mask( structure, annotation, df, fg = 1 ) :
    
    slist = df.loc[structure]['substructures']
    slist = slist.split('/')
    slist = [ int(x) for x in slist ]
    
    arr = sitk.GetArrayFromImage( annotation )
    idx = np.isin(arr,slist)
    output = np.zeros(arr.shape,np.uint16)
    output[idx] = fg
    
    output_image = sitk.GetImageFromArray( output )
    output_image.CopyInformation( annotation )    
    return output_image

In [5]:
def overlay( base, image ) :
    
    barr = sitk.GetArrayFromImage( base )
    iarr = sitk.GetArrayFromImage( image )
    
    idx = np.where( iarr > 0 )
    barr[idx] = iarr[idx]
    
    output_image = sitk.GetImageFromArray( barr )
    output_image.CopyInformation( base )    
    return output_image

In [6]:
def set_foreground_value( image, fg ) :
    
    arr = sitk.GetArrayFromImage( image )
    idx = np.where( arr > 0 )
    arr[idx] = fg
    
    output_image = sitk.GetImageFromArray( arr )
    output_image.CopyInformation( image )    
    return output_image    

In [7]:
def interface( image1, image2, fg ) :

    mask1 = set_foreground_value( image1, fg )
    mask2 = set_foreground_value( image2, fg )
    
    mask1 = sitk.BinaryDilate( mask1, (1,1,1), sitk.sitkBall, 0, fg )
    mask2 = sitk.BinaryDilate( mask2, (1,1,1), sitk.sitkBall, 0, fg )
    
    arr1 = sitk.GetArrayFromImage( mask1 )
    arr2 = sitk.GetArrayFromImage( mask2 )
    
    idx = np.logical_and( arr1 == fg, arr2  == fg )
    output = np.zeros(arr1.shape,np.uint16)
    output[idx] = fg
 
    output_image = sitk.GetImageFromArray( output )
    output_image.CopyInformation( mask1 )    
    return output_image

In [8]:
root = create_mask('root', annotation, df, fg = 2 )
grey = create_mask('grey', annotation, df, fg = 3 )
fiber_tracts = create_mask( 'fiber tracts', annotation, df, fg = 4)
VS = create_mask('VS', annotation, df, fg = 5 )

In [9]:
MB = create_mask('MB', annotation, df, fg = 3 )
CB = create_mask('CB', annotation, df, fg = 3 )
MB_CB_interface = interface( MB, CB, fg = 2 )

In [10]:
PAG = create_mask('PAG', annotation, df, fg = 3 )
CBX = create_mask('CBX', annotation, df, fg = 3 )
PAG_CBX_interface = interface( PAG, CBX, fg = 5 )

In [11]:
P = create_mask('P', annotation, df, fg = 3 )
P_CBX_interface = interface( P, CBX, fg = 5 )

In [12]:
MY = create_mask('MY', annotation, df, fg = 3 )
MY_CBX_interface = interface( MY, CBX, fg = 2 )

In [13]:
CTXpl = create_mask('CTXpl', annotation, df, fg = 3 )
MB_CTXpl_interface = interface( MB, CTXpl, fg = 2 )

In [14]:
TH = create_mask('TH', annotation, df, fg = 3 )
TH_CTXpl_interface = interface( TH, CTXpl, fg = 2 )

In [15]:
bsc = create_mask('bsc', annotation, df, fg = 4 )
bsc_as_root = set_foreground_value( bsc, fg = 2)

In [16]:
hbc = create_mask('hbc', annotation, df, fg = 4 )
hbc_as_VS = set_foreground_value( hbc, fg = 5)

In [17]:
fi = create_mask('fi', annotation, df, fg = 6 )
chpl = create_mask('chpl', annotation, df, fg = 7 )

In [18]:
V3 = create_mask('V3', annotation, df, fg = 5 )
V3_as_root = set_foreground_value( V3, fg = 2)

In [19]:
combined = overlay( root, grey )
combined = overlay( combined, MB_CB_interface )
combined = overlay( combined, PAG_CBX_interface )
combined = overlay( combined, P_CBX_interface )
combined = overlay( combined, MY_CBX_interface )
combined = overlay( combined, MB_CTXpl_interface )
combined = overlay( combined, TH_CTXpl_interface )
combined = overlay(combined,fiber_tracts)
combined = overlay( combined, bsc_as_root )
combined = overlay( combined, hbc_as_VS )
combined = overlay( combined, fi )
combined = overlay(combined,VS)
combined = overlay( combined, V3_as_root )
combined = overlay( combined, chpl )

In [20]:
sitk.WriteImage( combined, 'combined.nii.gz', True)